In [1]:
import pandas as pd

import tensorflow as tf
import keras

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import tree

from matplotlib import pyplot as plt

import openpyxl

import numpy as np
np.random.seed(42)

In [2]:
### Wczytanie i obróbka danych


#Wczytywanie danych
df_pracownicy = pd.read_excel("Zadanie - bazy danych.xlsx", sheet_name=0)
df_sklepy = pd.read_excel("Zadanie - bazy danych.xlsx", sheet_name=1)
df_pracownicy_testowi = pd.read_excel("Zadanie - bazy danych.xlsx", sheet_name=2)
df_do_predykcji = pd.read_excel("Zadanie - bazy danych.xlsx", sheet_name=3)


#Nazwy kolumn df_sklepy
df_sklepy_columns = list(df_sklepy.columns)

#Usuwanie pustych wartości w kwestii numerów sklepów
df_sklepy = df_sklepy[df_sklepy['Numer sklepu'].notna()]
df_pracownicy = df_pracownicy[df_pracownicy['Numer sklepu'].notna()]

#Zmiana 0.0 w df_sklepy w NaN, aby błędy/brak danych były oznaczane jednorodnie
for col in ["Wynagrodzenia", "Premie", "Produktywność", "Sprzedaż"]:
    df_sklepy[col] = df_sklepy[col].replace(0, np.nan)

#Obliczanie przepracowanego czasu (zmiennej objaśnianej)
df_pracownicy['Czas_pracy'] = (df_pracownicy['Data rozwiązania umowy'] - df_pracownicy['Data zatrudnienia']).dt.days
df_pracownicy = df_pracownicy.drop(["Data zatrudnienia", "Data rozwiązania umowy"], axis=1)

#Usuwanie pracowników, których numerów sklepów nie ma w df_sklepy
for i in df_pracownicy["Numer sklepu"]:
    if i not in df_sklepy["Numer sklepu"].unique():
        df_pracownicy = df_pracownicy.drop(df_pracownicy[df_pracownicy['Numer sklepu'] == i].index)

#Wiek zamiast daty urodzenia
df_pracownicy['Data urodzenia'] = ((pd.to_datetime('today') - df_pracownicy['Data urodzenia']).dt.days)/365
df_pracownicy = df_pracownicy.rename(columns={"Data urodzenia":"Wiek"})

#Płeć jako zmienna binarna
df_pracownicy = df_pracownicy.rename(columns={"Płeć":"Kobieta"})
df_pracownicy['Kobieta'] = df_pracownicy['Kobieta'].map({'kobieta': 1, 'mężczyzna': 0})

#Numer pracownika
df_pracownicy = df_pracownicy.drop("Numer pracownika", axis=1)


In [3]:
### Uzupełnienie pustych wartości w df_sklepy medianą

imputer = SimpleImputer(strategy="median")
df_sklepy = pd.DataFrame(imputer.fit_transform(df_sklepy), columns=df_sklepy_columns)

In [4]:
### Dataframe do podstaw modelu

df = pd.merge(df_pracownicy, df_sklepy, on="Numer sklepu")
df = df.drop(["Numer sklepu", "Powód rozwiązania umowy", 'Inicjatywa rozwiązania umowy'], axis=1)

df

Stanowisko  Etat Obywatelstwo  Kobieta       Wiek  Czas_pracy  \
0     Stanowisko 1  0.25      polskie        0  30.663014         236   
1     Stanowisko 2  1.00      polskie        0  26.457534         972   
2     Stanowisko 2  1.00      polskie        0  47.339726         227   
3     Stanowisko 2  1.00      polskie        1  27.139726         828   
4     Stanowisko 2  1.00      polskie        0  26.591781         653   
...            ...   ...          ...      ...        ...         ...   
7933  Stanowisko 2  1.00      polskie        0  19.506849         427   
7934  Stanowisko 2  1.00      polskie        1  23.884932         617   
7935  Stanowisko 2  0.50      polskie        0  30.871233         285   
7936  Stanowisko 2  0.50   ukraińskie        1  21.572603         110   
7937  Stanowisko 2  1.00      polskie        1  39.049315         194   

      Wynagrodzenia    Premie  Produktywność  Sprzedaż  Wskaźnik rotacji  
0          2.245487  0.019484       1.344030  1.434746          0.068966  
1          1.471635  0.137143       1.585318  1.527905          0.000000  
2          1.220149  0.098249       1.683952  1.647673          0.000000  
3          1.243676  0.095783       1.280112  1.022476          0.000000  
4          1.829930  0.365991       1.691277  1.985777          0.000000  
...             ...       ...            ...       ...               ...  
7933       2.245487  0.019484       1.344030  1.434746          0.068966  
7934       1.517370  0.031154       1.505348  1.187342          0.000000  
7935       2.245487  0.019484       1.344030  1.434746          0.068966  
7936       1.517370  0.031154       1.505348  1.187342          0.000000  
7937       1.517370  0.031154       1.505348  1.187342          0.000000  

[7938 rows x 11 columns]

In [5]:
# Dataframe tylko z wartościami numerycznymi
df_numerical = df.drop(["Stanowisko", "Obywatelstwo", "Kobieta", "Czas_pracy", "Etat"], axis=1)
df_numerical_columns = list(df_numerical) #Nazwy kolumn

In [6]:
### Pipeline standaryzujący i kodujący zmienne kategoryczne

#Standaryzacja (X-średnia)/odchylenie standardowe 
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])


num_attribs = df_numerical_columns #Kolumny z wartościami numerycznymi
cat_attribs = ["Obywatelstwo", "Stanowisko"] #Kolumny z wartościami kategorycznymi

full_pipeline = ColumnTransformer(
                transformers=[
                        ('pass', 'passthrough', ['Kobieta', "Czas_pracy", "Etat"]), #Nie wykonuje żadnych operacji
                        ("num", num_pipeline, num_attribs),
                        ("cat", OneHotEncoder(), cat_attribs)
                        ] 
        )

df = full_pipeline.fit_transform(df)
try:
        df = pd.DataFrame(df.toarray())
except:
        df = pd.DataFrame(df)

df

0      1     2         3         4         5         6         7   \
0     0.0  236.0  0.25 -0.694349  0.995076 -1.316954 -1.296652 -0.500076   
1     0.0  972.0  1.00 -1.052323 -0.523635 -0.160060  0.052478 -0.344901   
2     0.0  227.0  1.00  0.725189 -1.017186 -0.542495  0.603975 -0.145404   
3     1.0  828.0  1.00 -0.994254 -0.971014 -0.566740 -1.654041 -1.186790   
4     0.0  653.0  1.00 -1.040896  0.179531  2.090095  0.644934  0.417773   
...   ...    ...   ...       ...       ...       ...       ...       ...   
7933  0.0  427.0  1.00 -1.643972  0.995076 -1.316954 -1.296652 -0.500076   
7934  1.0  617.0  1.00 -1.271306 -0.433878 -1.202200 -0.394663 -0.912174   
7935  0.0  285.0  0.50 -0.676625  0.995076 -1.316954 -1.296652 -0.500076   
7936  1.0  110.0  0.50 -1.468133 -0.433878 -1.202200 -0.394663 -0.912174   
7937  1.0  194.0  1.00  0.019501 -0.433878 -1.202200 -0.394663 -0.912174   

            8    9   ...   31   32   33   34   35   36   37   38   39   40  
0     1.230366  0.0  ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1    -0.667882  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2    -0.667882  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3    -0.667882  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4    -0.667882  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
7933  1.230366  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
7934 -0.667882  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
7935  1.230366  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
7936 -0.667882  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
7937 -0.667882  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[7938 rows x 41 columns]

In [7]:
### Podział na train i test sets

train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

# Labels
train_set_label = train_set[1]
train_set = train_set.drop(1, axis=1)

test_set_label = test_set[1]
test_set = test_set.drop(1, axis=1)

### Nearest Neighbors Regression

In [8]:
neigh = KNeighborsRegressor(n_neighbors=5, weights='distance')
neigh.fit(train_set, train_set_label)

predictions_neigh = neigh.predict(test_set)

neigh_mae = mean_absolute_error(test_set_label, predictions_neigh)
neigh_mae

274.43147405367273

In [9]:
predictions_neigh = pd.DataFrame(predictions_neigh)
test_set_label = pd.DataFrame(test_set_label)
test_set_label = test_set_label.reset_index(drop=True)

df_porównanie0 = predictions_neigh.join(test_set_label)
df_porównanie0

0      1
0     341.934549  698.0
1     290.631201  328.0
2     522.926604  922.0
3     478.467379  111.0
4     646.169451  339.0
...          ...    ...
1583  898.511270  639.0
1584  324.881458  867.0
1585  644.030802  263.0
1586  507.553969  664.0
1587  622.084192  940.0

[1588 rows x 2 columns]

### Neural network

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu', input_shape=[train_set.shape[1]]),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=8, activation='linear'),
    tf.keras.layers.Dense(units=1)
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                1312      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1985 (7.75 KB)
Trainable params: 1985 (7.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,     
              loss='mean_absolute_error')

In [12]:
losses = model.fit(train_set, train_set_label,
                   validation_data=(test_set, test_set_label),
                   batch_size=512, 
                   epochs=40,
                   )

Epoch 1/40

13/13 [==============================] - 2s 35ms/step - loss: 492.8149 - val_loss: 495.6819
Epoch 2/40
13/13 [==============================] - 0s 5ms/step - loss: 492.3911 - val_loss: 495.2519
Epoch 3/40
13/13 [==============================] - 0s 4ms/step - loss: 491.9018 - val_loss: 494.6421
Epoch 4/40
13/13 [==============================] - 0s 3ms/step - loss: 491.1251 - val_loss: 493.6091
Epoch 5/40
13/13 [==============================] - 0s 5ms/step - loss: 489.7860 - val_loss: 491.8263
Epoch 6/40
13/13 [==============================] - 0s 5ms/step - loss: 487.4878 - val_loss: 488.8102
Epoch 7/40
13/13 [==============================] - 0s 5ms/step - loss: 483.6802 - val_loss: 483.9216
Epoch 8/40
13/13 [==============================] - 0s 4ms/step - loss: 477.6626 - val_loss: 476.3431
Epoch 9/40
13/13 [==============================] - 0s 4ms/step - loss: 468.5990 - val_loss: 465.1678
Epoch 10/40
13/13 [==============================] - 0s 4ms/step - loss: 455.665

In [13]:
predictions_NN = pd.DataFrame(model.predict(test_set))
test_set_label = pd.DataFrame(test_set_label)
test_set_label = test_set_label.reset_index(drop=True)

df_porównanie2 = predictions_NN.join(test_set_label)
df_porównanie2

50/50 [==============================] - 0s 1ms/step


0      1
0     486.240784  698.0
1     484.699097  328.0
2     432.090240  922.0
3     463.266785  111.0
4     529.405884  339.0
...          ...    ...
1583  556.800659  639.0
1584  514.473938  867.0
1585  526.582703  263.0
1586  409.570435  664.0
1587  499.219177  940.0

[1588 rows x 2 columns]

### Random Forest Regressor

In [14]:
### Decision forest
forest_reg = RandomForestRegressor(random_state=42, 
                                   n_estimators=100,
                                   criterion='squared_error',
                                   max_depth=None, 
                                   min_samples_split=4)
forest_reg.fit(train_set, train_set_label)

predictions_forest = forest_reg.predict(test_set)

forest_mae = mean_absolute_error(test_set_label, predictions_forest)
forest_mae

259.09843897403675

In [15]:
predictions_forest = pd.DataFrame(predictions_forest)
test_set_label = pd.DataFrame(test_set_label)
test_set_label = test_set_label.reset_index(drop=True)

df_porównanie1 = predictions_forest.join(test_set_label)
df_porównanie1

0      1
0     362.643714  698.0
1     526.156984  328.0
2     555.941810  922.0
3     300.925008  111.0
4     558.292940  339.0
...          ...    ...
1583  690.315286  639.0
1584  408.876611  867.0
1585  454.551667  263.0
1586  506.340512  664.0
1587  494.312583  940.0

[1588 rows x 2 columns]

### Ostateczna predykcja

In [16]:
### Standaryzacja 
# df_do_predykcji["Wskaźnik rotacji"] = 0.06
df_do_predykcji["Wiek"] = (df_do_predykcji["Wiek"]-df_do_predykcji["Wiek"].mean())/df_do_predykcji["Wiek"].std()
df_do_predykcji["Wynagrodzenia"] = (df_do_predykcji["Wynagrodzenia"]-df_do_predykcji["Wynagrodzenia"].mean())/df_do_predykcji["Wynagrodzenia"].std()
df_do_predykcji["Premie"] = (df_do_predykcji["Premie"]-df_do_predykcji["Premie"].mean())/df_do_predykcji["Premie"].std()
df_do_predykcji["Produktywność"] = (df_do_predykcji["Produktywność"]-df_do_predykcji["Produktywność"].mean())/df_do_predykcji["Produktywność"].std()
df_do_predykcji["Sprzedaż"] = (df_do_predykcji["Sprzedaż"]-df_do_predykcji["Sprzedaż"].mean())/df_do_predykcji["Sprzedaż"].std()
df_do_predykcji["Wskaźnik rotacji"] = (df_do_predykcji["Wskaźnik rotacji"]-df_do_predykcji["Wskaźnik rotacji"].mean())/df_do_predykcji["Wskaźnik rotacji"].std()

In [17]:
df_do_predykcji

Kobieta  Etat      Wiek  Wynagrodzenia    Premie  Produktywność  Sprzedaż  \
0        1   1.0  0.647098       0.910680  0.916506       1.026948  0.999288   
1        1   1.0  1.727323      -0.825472 -0.036735      -1.706894 -1.854483   
2        1   1.0 -0.682618      -0.899518  0.693344       1.422470  0.781089   
3        1   1.0 -0.695304      -0.965507 -1.468415      -0.811788 -0.455520   
4        1   1.0  0.903507       0.312673  0.006388      -0.152681  0.321950   
5        1   1.0  0.379924      -0.544325 -0.663625      -0.939799 -0.999765   
6        1   1.0 -1.257713       2.020934  1.156966      -0.399703  1.100684   
7        1   0.5  0.726289      -0.367810 -1.694835       0.474265 -0.575006   
8        1   1.0 -0.716832       0.910680  0.916506       1.026948  0.999288   
9        1   1.0 -1.031673      -0.552334  0.173900       0.060234 -0.317526   

   Wskaźnik rotacji  polskie  ukraińskie  ...  syryjskie  Stanowisko 1  \
0         -0.316228        1           0  ...          0             0   
1         -0.316228        1           0  ...          0             0   
2         -0.316228        1           0  ...          0             0   
3         -0.316228        1           0  ...          0             0   
4         -0.316228        1           0  ...          0             0   
5         -0.316228        1           0  ...          0             0   
6         -0.316228        1           0  ...          0             0   
7         -0.316228        1           0  ...          0             0   
8         -0.316228        1           0  ...          0             0   
9          2.846050        1           0  ...          0             0   

   Stanowisko 2  Stanowisko 3  Stanowisko 5  Stanowisko 4  Stanowisko 6  \
0             1             0             0             0             0   
1             1             0             0             0             0   
2             0             0             1             0             0   
3             0             0             0             1             0   
4             1             0             0             0             0   
5             0             0             1             0             0   
6             0             0             1             0             0   
7             0             1             0             0             0   
8             0             0             1             0             0   
9             0             0             0             1             0   

   Stanowisko 7  Stanowisko 8  Stanowisko 9  
0             0             0             0  
1             0             0             0  
2             0             0             0  
3             0             0             0  
4             0             0             0  
5             0             0             0  
6             0             0             0  
7             0             0             0  
8             0             0             0  
9             0             0             0  

[10 rows x 40 columns]

In [18]:
# #Forest model wizualnie
# train_set.columns = ["Kobieta", "Etat", "Wiek", "Wynagrodzenia", "Premie", "Produktywność", "Sprzedaż", "Wskaźnik rotacji", "polskie", "ukraińskie", "rosyjskie", "białoruskie", "gruzińskie", "uzbeckie", "litewskie", "kazachstańskie", "bułgarskie", "mołdawskie", "tadżyckie", "rumuńskie", "hinduskie", "hiszpańskie", "armeńskie", "egipskie", "jordańskie", "słowackie", "marokańskie", "gwinejskie", "tunezyjskie", "irańskie", "syryjskie", "Stanowisko 1", "Stanowisko 2", "Stanowisko 3", "Stanowisko 5", "Stanowisko 4", "Stanowisko 6", "Stanowisko 7", "Stanowisko 8", "Stanowisko 9"]
# plt.figure(figsize=(20,20))
# _ = tree.plot_tree(forest_reg.estimators_[0], feature_names=train_set.columns, filled=True)

In [19]:
# odpowiedź = pd.DataFrame(neigh.predict(df_do_predykcji))
# odpowiedź = pd.DataFrame(model.predict(df_do_predykcji))
odpowiedź = pd.DataFrame(forest_reg.predict(df_do_predykcji))
print(odpowiedź[0].mean())
odpowiedź

514.0650726551227


c:\Users\barte\anaconda3\envs\Python_3_10_13_Tensorflow\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


0
0  404.004488
1  537.758262
2  471.730452
3  483.209988
4  577.553333
5  498.567798
6  470.416996
7  595.510883
8  516.261607
9  585.636919

In [20]:
### Odpowiedź w formie dataframe

df_pracownicy_testowi["Przewidywana_data_odejścia"] = df_pracownicy_testowi["Data zatrudnienia"] + pd.to_timedelta(odpowiedź[0], unit='d')
df_pracownicy_testowi["Przewidywana_data_odejścia"] = df_pracownicy_testowi["Przewidywana_data_odejścia"].dt.date

df_pracownicy_testowi

Numer pracownika Data zatrudnienia    Stanowisko  Numer sklepu  Etat  \
0          90000001        2007-10-17  Stanowisko 2          4949   1.0   
1          90000002        2007-10-23  Stanowisko 2          7263   1.0   
2          90000003        2007-11-05  Stanowisko 5          3535   1.0   
3          90000004        2007-11-06  Stanowisko 4          4442   1.0   
4          90000005        2024-02-01  Stanowisko 2          5939   1.0   
5          90000006        2008-02-01  Stanowisko 5          4661   1.0   
6          90000007        2008-02-08  Stanowisko 5          2968   1.0   
7          90000008        2017-04-11  Stanowisko 3          1981   0.5   
8          90000009        2008-03-17  Stanowisko 5          4949   1.0   
9          90000010        2008-04-15  Stanowisko 4          1244   1.0   

  Obywatelstwo     Płeć Data urodzenia Przewidywana_data_odejścia  
0      polskie  kobieta     1975-04-05                 2008-11-24  
1      polskie  kobieta     1967-07-26                 2009-04-12  
2      polskie  kobieta     1984-09-23                 2009-02-18  
3      polskie  kobieta     1984-10-26                 2009-03-03  
4      polskie  kobieta     1973-06-07                 2025-08-31  
5      polskie  kobieta     1977-02-28                 2009-06-13  
6      polskie  kobieta     1988-10-28                 2009-05-23  
7      polskie  kobieta     1974-09-11                 2018-11-27  
8      polskie  kobieta     1984-12-21                 2009-08-15  
9      polskie  kobieta     1987-03-20                 2009-11-21